In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pickle
import numpy as np 

Loading & Preprocessing Dataset

In [ ]:
X_Temp = open('X_Data','rb')
X_Data = pickle.load(X_Temp)

Y_Temp = open('Y_Data','rb')
Y_Data = pickle.load(Y_Temp)

X_Data = np.array(X_Data)
Y_Data = np.array(Y_Data)

print(X_Data.shape)
print(Y_Data.shape)


X_train,X_test,y_train,y_test = train_test_split(X_Data,Y_Data,test_size=0.2,random_state=42,shuffle=True)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

#for X,y in zip(X_test,y_test):
    #plt.imshow(X,label=y)
    #print(y)
    #plt.legend()
    #plt.show()


# pickle.dump(scaler, open('scaler.pkl', 'wb'))


# datagen = ImageDataGenerator(
#     featurewise_center=True,
#     featurewise_std_normalization=True,
#     zoom_range = (0.95,0.99),
#     rotation_range=25,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     horizontal_flip=True,
#     validation_split=0.2
# )

# datagen.fit(X_train,augment=True)

# print(X_train.mean())
# print(X_train.std())
# print(X_train[0])

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train.reshape(X_train.shape[0],-1))
X_test = scaler.transform(X_test.reshape(X_test.shape[0],-1))

X_train = X_train.reshape(-1,224,224,1) 
X_test = X_test.reshape(-1,224,224,1) 

print(X_train.mean())
print(X_train.std())

#for X_batch, y_batch in datagen.flow(X_train, y_train, batch_size=9):
    #for i in range(0, 9):
        #plt.subplot(330 + 1 + i)
        #plt.imshow(X_batch[i])
        #plt.show()
    #break


Creating Model 

In [ ]:
#------------------ Bloc MobileNet -----------------------

class TikaiNetBlock(layers.Layer):
    
    def __init__(self,filters,strides):
        super(TikaiNetBlock,self).__init__()
        self.dw = layers.DepthwiseConv2D(kernel_size=3,strides=strides,padding="same")
        self.bn = layers.BatchNormalization()
        self.bn_2 = layers.BatchNormalization()
        self.relu = layers.ReLU()
        self.relu_2 = layers.ReLU()
        self.conv_2 = layers.Conv2D(filters=filters,kernel_size=1,strides=1)
    
    def call(self,X):
        
        X = self.dw(X)
        X = self.bn(X)
        X = self.relu(X)

        X = self.conv_2(X)
        X = self.bn_2(X)
        X = self.relu_2(X)

        return X



Compiling & Training & Saving

In [ ]:

model = Sequential()
model.add(layers.Conv2D(32,3,2,padding="same",input_shape=(224,224,1)))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())

model.add(TikaiNetBlock(64,1))
model.add(TikaiNetBlock(128,2))
model.add(TikaiNetBlock(128,1))
model.add(TikaiNetBlock(256,2))
model.add(TikaiNetBlock(256,1))
model.add(TikaiNetBlock(512,2))
for _ in range(5):
    model.add(TikaiNetBlock(512,1))

model.add(TikaiNetBlock(1024,2))
model.add(TikaiNetBlock(1024,1))
model.add(layers.AveragePooling2D(pool_size=7,strides=1))
model.add(layers.Flatten())
model.add(layers.Dense(2,activation="softmax"))



model.compile(optimizer= Adam(),
              loss=SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

BS = 32
EPOCHS = 20

# history = model.fit(
#     datagen.flow(X_train, y_train, batch_size=32,
#     subset='training'),
#     validation_data=datagen.flow(X_train, y_train,
#     batch_size=8, subset='validation'),
#     epochs=EPOCHS
# )
history = model.fit(X_train,y_train,batch_size=BS,epochs=EPOCHS,validation_data=(X_test,y_test))
                    
model.save("mask_model",save_format="tf") 
#A = model.evaluate(X_test,y_test)
#print(A)

In [ ]:
print(history.history.keys())

val_loss_curve =  history.history["val_loss"]
val_acc_curve = history.history["val_accuracy"]
acc_curve = history.history["accuracy"]
loss_curve = history.history["loss"]

plt.figure()

plt.subplot(2,1,1)
plt.plot(val_acc_curve,c="red",label="Val_Acc")
plt.plot(acc_curve,c="blue",label="Accuracy")
plt.legend()

plt.subplot(2,1,2)
plt.plot(val_loss_curve,c="red",label="Val_Loss")
plt.plot(loss_curve,c="blue",label="Loss")
plt.legend()

plt.savefig("plot.png")
plt.show()




In [ ]:

A = model.predict(X_test[600].reshape(1,224,224,1))
print(A*100)
plt.imshow(X_test[600])